# Mr Spwidwer 0w0

Student Name: Kaitlin Jarboe

Date: 3/26/20 (modified from 12/28/18 version)

IDE version: Jupyter Notebook 5.74
    Python 3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)]

OS version: Windows 10

Lab Assignment: Ch. 10-12 Project

 
# Analysis
					 
	Inputs (data description your app accepts):
        None
	
	Outputs (data description data your program produces and in what formats):
        - Marvel wikia scrape results (.csv)
        - DC wikia scrape results (.csv)


# Design

1. Setup and clean for spider
    1. notebook settings
    2. remove old files if necessary
2. Import scrapy
3. Setup Json
    1. import json
    2. define
4. Setup scrapy
    1. import csv
    2. import logging
    3. define scrapy class object
5. Create Dc spider
    1. initialize
        1. allowed domains and starting url
        2. customize pipelines
    2. define the function to scrape a character page
        - url, current alias, real name,
        identity, occupation, citizenship, marital status,
        eye color, hair color, sex,
        first appearance, first date, last appearance, last date,
        universe, creator(s),
        gender/sexual minority, living status, number of appearances, alignment

In [1]:
# Settings for notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Show Python version
import platform
platform.python_version()

# set cwd variable to current working directory
import os
cwd = os.getcwd()

# clear directory
try:
    os.remove(cwd+'\dccomicspider.jl')
except: pass
try:
    os.remove(cwd+'\Data\dc2020.csv')
except: pass
try:
    os.remove(cwd+'\m2020.csv')
except: pass

In [2]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

In [3]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('dccomicspider.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [4]:
import csv
from pprint import pprint

import logging
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from scrapy.selector import Selector

class DmozItem(scrapy.Item):
    # define the fields for your item here like:
    link = scrapy.Field()
    attr = scrapy.Field()

In [5]:
class DCProfileSpider(CrawlSpider):
    name = "dcprofile"
#    allowed_domains = ['fandom.com']
    allowed_domains = ['fandom.com','wikia.com']
    start_urls = ['http://dc.wikia.com/wiki/Category:Characters']
        
    custom_settings = {
        'LOG_FILE': 'dclog',
        'LOG_LEVEL': logging.INFO,
        'LOG_ENABLED': True,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'csv',                                 # Used for pipeline 2
        'FEED_URI': 'dc2020.csv'                        # Used for pipeline 2
    }
    
    
    def parse_character(self, response):
        def get_info(data, index):
            if index == 0:
                info = response.xpath('//*[@id="mw-content-text"]/aside/div[child::h3[contains(.,$data)]]/div/a/text()',data=data).extract_first()
#                 //*[@id="mw-content-text"]/aside/div[2]/div
                if not info:
                    info = response.xpath('//*[@id="mw-content-text"]/aside/div[child::h3[contains(.,$data)]]/div/text()',data=data).extract_first()
            else:
                info = response.xpath('//*[@id="mw-content-text"]/aside/section[child::h2[contains(.,$index)]]/div[child::h3[contains(.,$data)]]/div/a/text()',data=data,index=index).extract_first()
                if not info:
                    info = response.xpath('//*[@id="mw-content-text"]/aside/section[child::h2[contains(.,$index)]]/div[child::h3[contains(.,$data)]]/div/text()',data=data,index=index).extract_first()
            if not info: info = ' '
            return info
        
#         Alive
#         //*[@id="articleCategories"]/div/ul/li[1]/span/a
        ali = response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Deceased Characters"]/span/a/text()').extract_first()
        if not ali:
            ali = 'Living Characters'
    
#         GSM
# //*[@id="articleCategories"]/div/ul/li[31]/span/a/text()
        gsm = [response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Bisexual Characters"]/span/a/text()').extract_first(),
              response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Asexual Characters"]/span/a/text()').extract_first(),
              response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Homosexual Characters"]/span/a/text()').extract_first(),
              response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Pansexual Characters"]/span/a/text()').extract_first(),
              response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Genderless Characters"]/span/a/text()').extract_first(),
              response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Transgender Characters"]/span/a/text()').extract_first()]
        gsm_final = []
        for i in range(0,len(gsm)):
            if gsm[i] != None:
                gsm_final.append(gsm[i])
        if gsm_final == []: gsm_final = ' '
        
#         //*[@id="mw-content-text"]/ul[2]/li[1]/a
        appearances_var = response.xpath('//*[@id="mw-content-text"]/h2[descendant::text()="Related"]/following::ul/li[1]/a/text()').extract_first()
        try:
            num_appearances = appearances_var.split(' ',1)[0]
            num_appearances = int(num_appearances.replace(',',''))
        except: num_appearances = 1
            
        cur = get_info("Current Alias", 0)
        if cur == ' ' and get_info("Identity", "Status") == 'No Dual Identity':
            cur = 'N/A'
            
        yield {
            'Url': response.css('head link::attr(href)').extract_first(),
            'Current_Alias': cur, 'Real_Name': get_info("Real Name", 0),
            
            'IdState': get_info("Identity", "Status"), 'Occupation': get_info("Occupation", "Status"), 'Citizenship': get_info("Citizenship", "Status"), 'MaritalStatus': get_info("Marital Status", "Status"),

            'Eye': get_info("Eyes", "Characteristics"), 'Hair': get_info("Hair", "Characteristics"), 'Sex': get_info("Gender", "Characteristics"),
            
#             //*[@id="mw-content-text"]/aside/section[4]/table/tbody/tr/td[1]/div
            'FAppearance': response.xpath('//*[@id="mw-content-text"]/aside/section[4]/table/tbody/tr/td[1]/div/b/a/@title').extract_first(),
            'FDate': response.xpath('//*[@id="mw-content-text"]/aside/section[4]/table/tbody/tr/td[1]/div/a/text()').extract_first(),
            'LAppearance': response.xpath('//*[@id="mw-content-text"]/aside/section[4]/table/tbody/tr/td[2]/div/b/a/@title').extract_first(),
            'LtDate': response.xpath('//*[@id="mw-content-text"]/aside/section[4]/table/tbody/tr/td[2]/div/a/text()').extract_first(),
            
            'Universe': get_info("Universe", "Origin"), 'Creators': get_info("Creators", "Origin"),

            # Alignment 
            'GSM': gsm_final,
            'Alive': ali, 'Appearances': num_appearances,
            'Alignment': get_info("Alignment", "Status"),
        }
        
    def parse(self, response):
        next_page = Selector(response=response.copy()).css('a.category-page__pagination-next::attr(href)').extract_first()
        self.logger.info(next_page)
        
        for quote in response.css('li.category-page__member'):
                if 'Category:' not in quote.xpath('./a/text()').extract_first():
                    yield response.follow(
                        ('http://dc.wikia.com'+quote.css('a::attr(href)').extract_first()),
                        self.parse_character)
        if next_page is not None:
            yield response.follow(next_page, self.parse)

In [6]:
class MarvelProfileSpider(CrawlSpider):
    name = "marvelprofile"
#    allowed_domains = ['fandom.com']
    allowed_domains = ['fandom.com','wikia']
    start_urls = ['http://marvel.fandom.com/wiki/Category:Characters']
        
    custom_settings = {
        'LOG_FILE': 'marvellog',
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'csv',                                 # Used for pipeline 2
        'FEED_URI': 'm2020.csv'                        # Used for pipeline 2
    }
    
    
    def parse_character(self, response):
        def get_info(data, index):
            if index == 0:
                info = response.xpath('//*[@id="mw-content-text"]//aside/div[child::h3[contains(.,$data)]]/div/a/text()',data=data).extract_first()
#                 //*[@id="mw-content-text"]/aside/div[2]/div
                if not info:
                    info = response.xpath('//*[@id="mw-content-text"]//aside/div[child::h3[contains(.,$data)]]/div/text()',data=data).extract_first()
            else:
                info = response.xpath('//*[@id="mw-content-text"]//aside/section[child::h2[contains(.,$index)]]/div[child::h3[contains(.,$data)]]/div/a/text()',data=data,index=index).extract_first()
                if not info:
                    info = response.xpath('//*[@id="mw-content-text"]//aside/section[child::h2[contains(.,$index)]]/div[child::h3[contains(.,$data)]]/div/text()',data=data,index=index).extract_first()
            if not info: info = ' '
            return info
        
#         Alive
#         //*[@id="articleCategories"]/div/ul/li[1]/span/a
        ali = response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Deceased Characters"]/span/a/text()').extract_first()
        if not ali:
            ali = 'Living Characters'
    
#         GSM
# //*[@id="articleCategories"]/div/ul/li[31]/span/a/text()
        gsm = [response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Bisexual Characters"]/span/a/text()').extract_first(),
              response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Asexual Characters"]/span/a/text()').extract_first(),
              response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Homosexual Characters"]/span/a/text()').extract_first(),
              response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Pansexual Characters"]/span/a/text()').extract_first(),
              response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Genderless Characters"]/span/a/text()').extract_first(),
              response.xpath('//*[@id="articleCategories"]/div/ul/li[@data-name="Transgender Characters"]/span/a/text()').extract_first()]
        gsm_final = []
        for i in range(0,len(gsm)):
            if gsm[i] != None:
                gsm_final.append(gsm[i])
        if gsm_final == []: gsm_final = ' '
        
#         //*[@id="mw-content-text"]/ul[2]/li[1]/a
        appearances_var = response.xpath('//*[@id="mw-content-text"]/h2[descendant::text()="Links and References"]/following::ul/li[1]/a/text()').extract_first()
        try:
            num_appearances = appearances_var.split(' ',1)[0]
            num_appearances = int(num_appearances.replace(',',''))
        except: num_appearances = 1
            
        cur = get_info("Current Alias", 0)
        if cur == ' ' and get_info("Identity", "Status") == 'No Dual Identity':
            cur = 'N/A'
            
        yield {
            'Url': response.css('head link::attr(href)').extract_first(),
            'Current_Alias': cur, 'Real_Name': get_info("Real Name", 0),
            
            'IdState': get_info("Identity", "Status"), 'Occupation': get_info("Occupation", "Status"), 'Citizenship': get_info("Citizenship", "Status"), 'MaritalStatus': get_info("Marital Status", "Status"),

            'Eye': get_info("Eyes", "Characteristics"), 'Hair': get_info("Hair", "Characteristics"), 'Sex': get_info("Gender", "Characteristics"),
            
#             //*[@id="mw-content-text"]/aside/section[4]/table/tbody/tr/td[1]/div
            'FAppearance': response.xpath('//*[@id="mw-content-text"]/aside//table/tbody/tr/td/div/b/a/@title').extract_first(),
            'FDate': response.xpath('//*[@id="mw-content-text"]/aside/section[4]/table/tbody/tr/td[1]/div/a/text()').extract_first(),
            'LAppearance': response.xpath('//*[@id="mw-content-text"]/aside/section[4]/table/tbody/tr/td[2]/div/b/a/@title').extract_first(),
            'LtDate': response.xpath('//*[@id="mw-content-text"]/aside/section[4]/table/tbody/tr/td[2]/div/a/text()').extract_first(),
            
            'Universe': get_info("Universe", "Origin"), 'Creators': get_info("Creators", "Origin"),

            # Alignment 
            'GSM': gsm_final,
            'Alive': ali, 'Appearances': num_appearances,
            'Alignment': get_info("Alignment", "Status"),
        }
        
    def parse(self, response):
        next_page = Selector(response=response.copy()).css('a.category-page__pagination-next::attr(href)').extract_first()
        self.logger.info(next_page)
        
        for quote in response.css('li.category-page__member'):
                if 'Category:' not in quote.xpath('./a/text()').extract_first():
                    yield response.follow(
                        ('http://marvel.fandom.com'+quote.css('a::attr(href)').extract_first()),
                        self.parse_character)
                    
        if next_page is not None:
            yield response.follow(next_page, self.parse)

In [7]:
currentspider = MarvelProfileSpider

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(currentspider)
process.start()

2020-03-26 22:32:16 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2020-03-26 22:32:16 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.19.0, Twisted 18.9.0, Python 3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.1.1a  20 Nov 2018), cryptography 2.4.2, Platform Windows-10-10.0.18362-SP0
2020-03-26 22:32:16 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'csv', 'FEED_URI': 'm2020.csv', 'LOG_FILE': 'marvellog', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


In [16]:
import pandas as pd

df = pd.read_csv(cwd+'\m2020.csv')
df.drop_duplicates()
df.sort_values('Url')
df[df['Url'] != 'https://marvel.fandom.com/wiki/Character_Index']
df.to_csv(cwd+'\m2020.csv')

,Url,Current_Alias,Real_Name,IdState,Occupation,Citizenship,MaritalStatus,Eye,Hair,Sex,FAppearance,FDate,LAppearance,LtDate,Universe,Creators,GSM,Alive,Appearances,Alignment
0,https://marvel.fandom.com/wiki/Character_Index,,,,,,,,,,NaN,NaN,NaN,NaN,,,,Living Characters,1,
1,https://marvel.fandom.com/wiki/302_(Legion_Per...,302,Unknown,Secret Identity,,,Single,,Black,Male,NaN,NaN,NaN,NaN,Earth-616,Mike Carey,,Living Characters,2,
2,https://marvel.fandom.com/wiki/89P13_(Earth-12...,Rocket Raccoon,Rocket Racoon,No Dual Identity,Guardian of the Galaxy,Halfworlders,Single,Brown,"Black, Brown,",Male,NaN,NaN,NaN,NaN,Earth-12041,Tim Maltby,,Living Characters,95,
3,https://marvel.fandom.com/wiki/8-Ball_(Hobgobl...,8-Ball,Unknown,Secret Identity,Criminal,American,Single,,,Male,NaN,NaN,NaN,NaN,Earth-616,Dan Slott,,Living Characters,25,
4,https://marvel.fandom.com/wiki/89P13_(Earth-19...,Rocket,89P13,Public Identity,Adventurer; former mercenary,,Single,Brown,Black,Male,NaN,NaN,NaN,NaN,Earth-199999,James Gunn,,Living Characters,15,
5,https://marvel.fandom.com/wiki/803_(Earth-616),NaN,803,No Dual Identity,,,Single,,,Agender,NaN,NaN,NaN,NaN,Earth-616,Robbie Thompson,,Living Characters,11,
6,https://marvel.fandom.com/wiki/8-Ball_II_(Eart...,8-Ball,Unknown,Secret Identity,Professional criminal,American,Single,,,Male,NaN,NaN,NaN,NaN,Earth-616,Dan Slott,,Living Characters,1,
7,https://marvel.fandom.com/wiki/6-Ball_(Earth-616),6-Ball,Unknown,Secret Identity,Professional criminal; former henchman,American,Single,,,Male,NaN,NaN,NaN,NaN,Earth-616,Bob Budiansky,,Living Characters,1,
8,https://marvel.fandom.com/wiki/88_(Earth-616),88,Unknown,Secret Identity,Militia man,American,Single,Blue,Blond,Male,NaN,NaN,NaN,NaN,Earth-616,Fabian Nicieza,,Deceased Characters,3,
9,https://marvel.fandom.com/wiki/7-X9_(Earth-616),7-X9,Unknown,No Dual Identity,,,,,,Male,NaN,NaN,NaN,NaN,Earth-616,Chip Zdarsky,,Deceased Characters,1,


,Url,Current_Alias,Real_Name,IdState,Occupation,Citizenship,MaritalStatus,Eye,Hair,Sex,FAppearance,FDate,LAppearance,LtDate,Universe,Creators,GSM,Alive,Appearances,Alignment
31069,android-app://com.wikia.singlewikia.marvel/htt...,,Unknown,,,,,,,,NaN,NaN,NaN,NaN,,,,Living Characters,1,
58650,android-app://com.wikia.singlewikia.marvel/htt...,,Unknown,,,,,,,,NaN,NaN,NaN,NaN,,,,Living Characters,1,
48598,android-app://com.wikia.singlewikia.marvel/htt...,Wolverine,James Howlett,Public Identity,"Student, adventurer; formerly mercenary, gover...",Canadian,Single,Blue,Black,Male,NaN,NaN,NaN,NaN,Earth-1610,Mark Millar,,Living Characters,1,
28586,android-app://com.wikia.singlewikia.marvel/htt...,Peteparker,Nathan Elliott,,Scientist,American,,,,Male,NaN,NaN,NaN,NaN,,,,Living Characters,1,
67017,https://marvel.fandom.com/wiki/%22Spider-Girl%...,NaN,Unknown,No Dual Identity,Formerly student,,Single,Brown,Blond,Female,NaN,NaN,NaN,NaN,Earth-616,Christopher Yost,,Living Characters,1,
67016,https://marvel.fandom.com/wiki/%27Lectron_(Ear...,'Lectron,Unknown,Secret Identity,Adventurer,,,,,Male,NaN,NaN,NaN,NaN,Earth-12772,R.K. Sternsel,,Living Characters,1,
67015,https://marvel.fandom.com/wiki/%27Selka_(Earth...,'Selka,Unknown,Public Identity,Homeless,American,,,White,Female,NaN,NaN,NaN,NaN,Earth-928,Len Kaminski,,Living Characters,1,
67012,https://marvel.fandom.com/wiki/%27Spinner_(Ear...,,'Spinner,Secret Identity,,,Single,,,Male,NaN,NaN,NaN,NaN,Earth-616,Stuart Moore,,Living Characters,1,
67014,https://marvel.fandom.com/wiki/%27Spinner_(Ear...,'Spinner,Unknown,,"Government agent, police officer",,,,,Male,NaN,NaN,NaN,NaN,Earth-TRN461,Gameloft,,Living Characters,1,
12080,https://marvel.fandom.com/wiki/%C3%84kr%C3%A4s...,NaN,Äkräs,No Dual Identity,"Champion of the mortals, god of fertility",Taivas,,,Brown,Male,NaN,NaN,NaN,NaN,Earth-616,Anthony Flamini,,Living Characters,1,


,Url,Current_Alias,Real_Name,IdState,Occupation,Citizenship,MaritalStatus,Eye,Hair,Sex,FAppearance,FDate,LAppearance,LtDate,Universe,Creators,GSM,Alive,Appearances,Alignment
1,https://marvel.fandom.com/wiki/302_(Legion_Per...,302,Unknown,Secret Identity,,,Single,,Black,Male,NaN,NaN,NaN,NaN,Earth-616,Mike Carey,,Living Characters,2,
2,https://marvel.fandom.com/wiki/89P13_(Earth-12...,Rocket Raccoon,Rocket Racoon,No Dual Identity,Guardian of the Galaxy,Halfworlders,Single,Brown,"Black, Brown,",Male,NaN,NaN,NaN,NaN,Earth-12041,Tim Maltby,,Living Characters,95,
3,https://marvel.fandom.com/wiki/8-Ball_(Hobgobl...,8-Ball,Unknown,Secret Identity,Criminal,American,Single,,,Male,NaN,NaN,NaN,NaN,Earth-616,Dan Slott,,Living Characters,25,
4,https://marvel.fandom.com/wiki/89P13_(Earth-19...,Rocket,89P13,Public Identity,Adventurer; former mercenary,,Single,Brown,Black,Male,NaN,NaN,NaN,NaN,Earth-199999,James Gunn,,Living Characters,15,
5,https://marvel.fandom.com/wiki/803_(Earth-616),NaN,803,No Dual Identity,,,Single,,,Agender,NaN,NaN,NaN,NaN,Earth-616,Robbie Thompson,,Living Characters,11,
6,https://marvel.fandom.com/wiki/8-Ball_II_(Eart...,8-Ball,Unknown,Secret Identity,Professional criminal,American,Single,,,Male,NaN,NaN,NaN,NaN,Earth-616,Dan Slott,,Living Characters,1,
7,https://marvel.fandom.com/wiki/6-Ball_(Earth-616),6-Ball,Unknown,Secret Identity,Professional criminal; former henchman,American,Single,,,Male,NaN,NaN,NaN,NaN,Earth-616,Bob Budiansky,,Living Characters,1,
8,https://marvel.fandom.com/wiki/88_(Earth-616),88,Unknown,Secret Identity,Militia man,American,Single,Blue,Blond,Male,NaN,NaN,NaN,NaN,Earth-616,Fabian Nicieza,,Deceased Characters,3,
9,https://marvel.fandom.com/wiki/7-X9_(Earth-616),7-X9,Unknown,No Dual Identity,,,,,,Male,NaN,NaN,NaN,NaN,Earth-616,Chip Zdarsky,,Deceased Characters,1,
10,https://marvel.fandom.com/wiki/89P13_(Earth-91...,Rocket Raccoon,89P13,Public Identity,Adventurer,,Single,Brown,Black,Male,NaN,NaN,NaN,NaN,Earth-91119,The Amazing Society,,Living Characters,1,
